# Project 1


## Description

## Data import

In [198]:
import dlc_practical_prologue as prologue

# We import 1000 pairs of digits for the training and the testing inputs, targets and classes
number_of_pairs = 1000 
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets( 
    number_of_pairs)

# Quick verification of the sizes:
print('Size of train_input:', train_input.size())
print('Size of train_target:', test_target.size())
print('Size of train_classes:', test_classes.size())
print('Size of test_input:', test_input.size())
print('Size of test_target:', test_target.size())
print('Size of test_classes:', test_classes.size())

Size of train_input: torch.Size([1000, 2, 14, 14])
Size of train_target: torch.Size([1000])
Size of train_classes: torch.Size([1000, 2])
Size of test_input: torch.Size([1000, 2, 14, 14])
Size of test_target: torch.Size([1000])
Size of test_classes: torch.Size([1000, 2])


## CNN

In [199]:
import torch
from torch import nn
from torch.nn import functional as F

In [200]:
class ConvNet(nn.Module):
    def __init__(self, initial_layers=2, final_layers=2):
        super().__init__()
        self.conv1 = nn.Conv2d(initial_layers, 32, kernel_size=3)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(256, 200)
        self.batchnorm3 = nn.BatchNorm1d(200)
        self.fc2 = nn.Linear(200, final_layers)
        self.batchnorm4 = nn.BatchNorm1d(final_layers)
        #self.dropout = nn.Dropout(0.2)
        

    def forward(self, x):
        x = self.batchnorm1(F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2)))
        x = self.batchnorm2(F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2)))
        x = self.batchnorm3(F.relu(self.fc1(x.view(-1, 256))))
        x = self.batchnorm4(self.fc2(x))
        return x

## CNN with Weight Sharing

#### We define the same class wether we use auxiliary loss or not

In [201]:
class WeightSharing(nn.Module):
    def __init__(self, auxiliary_loss = False):
        super().__init__()
        self.sharedConvNet = ConvNet(initial_layers=1, final_layers=10)
        self.auxiliary_loss = auxiliary_loss

        self.fc1 = nn.Linear(20,100)
        self.fc2 = nn.Linear(100,20)
        self.fc3 = nn.Linear(20,2)
    
    def forward(self, x):
        image1 = x.narrow(1,0,1)
        image2 = x.narrow(1,1,1)

        output1 = self.sharedConvNet(image1)
        output2 = self.sharedConvNet(image2)
        outputCat = torch.cat((output1, output2), 1)

        outputCat = F.relu(self.fc1(outputCat))
        outputCat = F.relu(self.fc2(outputCat))
        outputCat = self.fc3(outputCat)

        
        if self.auxiliary_loss:
            return outputCat, output1, output2
        else:
            return outputCat

## Model training and number of errors

In [202]:
def train_model(model, train_input, train_target, mini_batch_size, nb_epochs = 25, lr = 5e-1, printAccLoss = False, auxiliary_loss = False, alpha = 0.3, beta = 0.5):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = lr)

    for e in range(nb_epochs):
        acc_loss = 0

        for b in range(0, train_input.size(0), mini_batch_size):
            if auxiliary_loss:
                trainTargetImage1 = (train_classes.narrow(1,0,1)).squeeze()
                trainTargetImage2 = (train_classes.narrow(1,1,1)).squeeze()

                outputCat, output1, output2 = model(train_input.narrow(0, b, mini_batch_size))
                lossCat = criterion(outputCat, train_target.narrow(0, b, mini_batch_size))
                loss1 = criterion(output1, trainTargetImage1.narrow(0, b, mini_batch_size))
                loss2 = criterion(output2, trainTargetImage2.narrow(0, b, mini_batch_size))

                loss = alpha*loss1 + alpha*loss2 + beta*lossCat

            else:
                output = model(train_input.narrow(0, b, mini_batch_size))
                loss = criterion(output, train_target.narrow(0, b, mini_batch_size))

            acc_loss = acc_loss + loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        if printAccLoss:
            print(e, acc_loss)

def compute_nb_errors(model, input, target, mini_batch_size, auxiliary_loss = False):
    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        if auxiliary_loss:
            outputCat, _, _ = model(input.narrow(0, b, mini_batch_size))
            _, predicted_classes = outputCat.max(1)
            for k in range(mini_batch_size):
                if target[b + k] != predicted_classes[k]:
                    nb_errors = nb_errors + 1
        else:
            output = model(input.narrow(0, b, mini_batch_size))
            _, predicted_classes = output.max(1)
            for k in range(mini_batch_size):
                if target[b + k] != predicted_classes[k]:
                    nb_errors = nb_errors + 1


    return nb_errors

## Models testing

In [203]:
import time
mini_batch_size = 50
number_of_runs = 10

def test_model(model_name, number_of_runs=10, print_times=False, print_accuracy=False, auxiliary_loss=False):
    train_error_ratios = []
    test_error_ratios = []
    for run in range(number_of_runs):
        if model_name == 'ConvNet':
            model = ConvNet()
        elif model_name == 'WeightSharing':
            model = WeightSharing()
        elif model_name == 'WeightSharingWithAuxiliaryLoss':
            model = WeightSharing(auxiliary_loss=True)
        else:
            raise ValueError('Please use one of the implemented methods: ConvNet, WeightSharing, WeightSharingWithAuxiliaryLoss')
            
        train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(number_of_pairs)

        if print_times: startTime = time.time()
        model.train(True)
        train_model(model, train_input, train_target, mini_batch_size, auxiliary_loss=auxiliary_loss)
        model.train(False)

        if print_times: endTime = time.time()

        nb_train_errors = compute_nb_errors(model, train_input, train_target, mini_batch_size,auxiliary_loss=auxiliary_loss)
        nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size, auxiliary_loss=auxiliary_loss)
        train_error_ratios.append(nb_train_errors/train_input.size(0))
        test_error_ratios.append(nb_test_errors/test_input.size(0))

        if print_accuracy:
            print('--- Run', run, '---')
            print('- train error ratio:', nb_train_errors/train_input.size(0))
            print('- test error ratio:', nb_test_errors/test_input.size(0))
        if print_times: print('- time for training:', str(endTime-startTime))
    return train_error_ratios, test_error_ratios


## Model tuning

### CNN

In [204]:
train_error_ratios, test_error_ratios = test_model('ConvNet', print_accuracy=True)

--- Run 0 ---
- train error ratio: 0.0
- test error ratio: 0.173
--- Run 1 ---
- train error ratio: 0.0
- test error ratio: 0.171
--- Run 2 ---
- train error ratio: 0.0
- test error ratio: 0.184
--- Run 3 ---
- train error ratio: 0.0
- test error ratio: 0.171
--- Run 4 ---
- train error ratio: 0.001
- test error ratio: 0.19
--- Run 5 ---
- train error ratio: 0.0
- test error ratio: 0.178
--- Run 6 ---
- train error ratio: 0.0
- test error ratio: 0.161
--- Run 7 ---
- train error ratio: 0.0
- test error ratio: 0.187
--- Run 8 ---
- train error ratio: 0.0
- test error ratio: 0.187
--- Run 9 ---
- train error ratio: 0.0
- test error ratio: 0.182


### CNN with Weight Sharing

In [205]:
train_error_ratios, test_error_ratios = test_model('WeightSharing', print_accuracy=True)

--- Run 0 ---
- train error ratio: 0.001
- test error ratio: 0.115
--- Run 1 ---
- train error ratio: 0.0
- test error ratio: 0.141
--- Run 2 ---
- train error ratio: 0.0
- test error ratio: 0.113
--- Run 3 ---
- train error ratio: 0.0
- test error ratio: 0.113
--- Run 4 ---
- train error ratio: 0.0
- test error ratio: 0.137
--- Run 5 ---
- train error ratio: 0.0
- test error ratio: 0.122
--- Run 6 ---
- train error ratio: 0.0
- test error ratio: 0.117
--- Run 7 ---
- train error ratio: 0.0
- test error ratio: 0.101
--- Run 8 ---
- train error ratio: 0.0
- test error ratio: 0.107
--- Run 9 ---
- train error ratio: 0.0
- test error ratio: 0.102


### CNN with Weight Sharing and Auxiliary Loss

In [206]:
train_error_ratios, test_error_ratios = test_model('WeightSharingWithAuxiliaryLoss', auxiliary_loss=True, print_accuracy=True)

--- Run 0 ---
- train error ratio: 0.0
- test error ratio: 0.195
--- Run 1 ---
- train error ratio: 0.006
- test error ratio: 0.189
--- Run 2 ---
- train error ratio: 0.017
- test error ratio: 0.22
--- Run 3 ---
- train error ratio: 0.032
- test error ratio: 0.226
--- Run 4 ---
- train error ratio: 0.016
- test error ratio: 0.224
--- Run 5 ---
- train error ratio: 0.017
- test error ratio: 0.212
--- Run 6 ---
- train error ratio: 0.005
- test error ratio: 0.171
--- Run 7 ---
- train error ratio: 0.013
- test error ratio: 0.233
--- Run 8 ---
- train error ratio: 0.003
- test error ratio: 0.213
--- Run 9 ---
- train error ratio: 0.029
- test error ratio: 0.209
